In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [24]:
#Hyperparameters
num_epochs = 20
batch_size = 4
learning_rate = 0.001

In [14]:
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [15]:
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms)
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

In [25]:
first = train_set[0]
features, labels = first
print(features, labels)
print(features.shape)

tensor([[[-0.5373, -0.6627, -0.6078,  ...,  0.2392,  0.1922,  0.1608],
         [-0.8745, -1.0000, -0.8588,  ..., -0.0353, -0.0667, -0.0431],
         [-0.8039, -0.8745, -0.6157,  ..., -0.0745, -0.0588, -0.1451],
         ...,
         [ 0.6314,  0.5765,  0.5529,  ...,  0.2549, -0.5608, -0.5843],
         [ 0.4118,  0.3569,  0.4588,  ...,  0.4431, -0.2392, -0.3490],
         [ 0.3882,  0.3176,  0.4039,  ...,  0.6941,  0.1843, -0.0353]],

        [[-0.5137, -0.6392, -0.6235,  ...,  0.0353, -0.0196, -0.0275],
         [-0.8431, -1.0000, -0.9373,  ..., -0.3098, -0.3490, -0.3176],
         [-0.8118, -0.9451, -0.7882,  ..., -0.3412, -0.3412, -0.4275],
         ...,
         [ 0.3333,  0.2000,  0.2627,  ...,  0.0431, -0.7569, -0.7333],
         [ 0.0902, -0.0353,  0.1294,  ...,  0.1608, -0.5137, -0.5843],
         [ 0.1294,  0.0118,  0.1137,  ...,  0.4431, -0.0745, -0.2784]],

        [[-0.5059, -0.6471, -0.6627,  ..., -0.1529, -0.2000, -0.1922],
         [-0.8431, -1.0000, -1.0000,  ..., -0

In [26]:
print(len(train_loader)) #iteration

12500


In [27]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [28]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # Input channels = 3 (RGB), Output channels (The number of filter) = 6, Filter (Kernel) size = 5
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = ConvNet()

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [30]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # original shape: [batch_size, 3, 32, 32] = [batch_size, 3, 1024]
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 1000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/20], Step [1000/12500], Loss: 1.8185
Epoch [1/20], Step [2000/12500], Loss: 2.1187
Epoch [1/20], Step [3000/12500], Loss: 0.9019
Epoch [1/20], Step [4000/12500], Loss: 1.5808
Epoch [1/20], Step [5000/12500], Loss: 1.6466
Epoch [1/20], Step [6000/12500], Loss: 0.7833
Epoch [1/20], Step [7000/12500], Loss: 1.4117
Epoch [1/20], Step [8000/12500], Loss: 1.0462
Epoch [1/20], Step [9000/12500], Loss: 1.6253
Epoch [1/20], Step [10000/12500], Loss: 1.0920
Epoch [1/20], Step [11000/12500], Loss: 1.5629
Epoch [1/20], Step [12000/12500], Loss: 2.2404
Epoch [2/20], Step [1000/12500], Loss: 1.5357
Epoch [2/20], Step [2000/12500], Loss: 0.5176
Epoch [2/20], Step [3000/12500], Loss: 0.5640
Epoch [2/20], Step [4000/12500], Loss: 1.5422
Epoch [2/20], Step [5000/12500], Loss: 1.3612
Epoch [2/20], Step [6000/12500], Loss: 0.9270
Epoch [2/20], Step [7000/12500], Loss: 0.8482
Epoch [2/20], Step [8000/12500], Loss: 0.7749
Epoch [2/20], Step [9000/12500], Loss: 0.9152
Epoch [2/20], Step [10000/12500

In [31]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    y_true = []
    y_pred = []
    for images, labels in test_loader:
        # images = images.to(device)
        # labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    
    
acc = 100.0 * n_correct / n_samples
print(f'Accuracy of the network on the 10000 test images: {acc}')

for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]} : {acc} %')
    
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")

# Classification report chi tiết từng lớp
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=classes))

Accuracy of the network on the 10000 test images: 59.65
Accuracy of plane : 68.7 %
Accuracy of car : 64.8 %
Accuracy of bird : 53.8 %
Accuracy of cat : 54.4 %
Accuracy of deer : 48.1 %
Accuracy of dog : 35.0 %
Accuracy of frog : 67.0 %
Accuracy of horse : 54.3 %
Accuracy of ship : 76.4 %
Accuracy of truck : 74.0 %
Precision: 0.6204
Recall:    0.5965
F1-score:  0.5987

Classification Report:
              precision    recall  f1-score   support

       plane       0.61      0.69      0.65      1000
         car       0.81      0.65      0.72      1000
        bird       0.42      0.54      0.47      1000
         cat       0.37      0.54      0.44      1000
        deer       0.56      0.48      0.52      1000
         dog       0.60      0.35      0.44      1000
        frog       0.69      0.67      0.68      1000
       horse       0.78      0.54      0.64      1000
        ship       0.72      0.76      0.74      1000
       truck       0.64      0.74      0.68      1000

    accura